In [1]:
from data import Datagenerator, TrainingStats
from copy import deepcopy
from model import LEO, load_model, save_model
from  torch.nn import MSELoss
from easydict import EasyDict as edict
from utils import load_config, check_experiment
from run import train_model
import torch.optim as optim
import argparse
import torch 
import torch.optim
import gc
from utils import display_data_shape, get_named_dict, one_hot_target,\
    softmax, sparse_crossentropy, calc_iou_per_class, log_data, load_config
import matplotlib.pyplot as plt
import numpy as np

c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Train

In [2]:
config = load_config()

In [3]:
leo, meta_data, classes = train_model(config)

num tasks: 2
tr_data shape: torch.Size([2, 192, 256, 40]),tr_data_masks shape: torch.Size([2, 384, 512]), val_data shape: torch.Size([5, 192, 256, 40]),val_masks shape: torch.Size([5, 384, 512])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [28, 14, 3, 3], but got 3-dimensional input of size [192, 256, 40] instead

## Predictions on Meta Val / Test

In [ ]:
leo

In [ ]:
dataloader = Datagenerator(config, dataset="pascal_voc", data_type="meta_val")
val_meta_data = dataloader.get_batch_data()
class_in_metadata = val_meta_data[-1]

In [ ]:
class_in_metadata

In [ ]:
select_class = -1
select_sample = 0

In [ ]:
print("class", class_in_metadata[select_class], "is selected")
batch_data = get_named_dict(val_meta_data, select_class)
input_data = batch_data.val_data[select_sample].permute(1, 2, 0)
plt.title("First three channels of embeddings is visualized")
plt.imshow(input_data.cpu().detach().numpy()[:,:,:3])

In [ ]:
batch_data.val_data.shape

In [ ]:
latents, kl_loss = leo.forward_encoder(batch_data.tr_data)
tr_loss, predicted_weights, _ = leo.forward_decoder(batch_data.tr_data, latents, batch_data.tr_data_masks)
val_loss = leo.finetuning_inner_loop(batch_data, tr_loss, predicted_weights)
_, _, predictions = leo.forward_decoder(batch_data.val_data, latents, batch_data.val_data_masks)

In [ ]:
select_sample = 0
def plot_batch_mask(batch_data, select_sample):
    input_mask = batch_data.val_data_masks[select_sample]
    plt.title("ground truth mask")
    plt.imshow(input_mask.cpu().detach().numpy(), cmap="gray")
    return

plot_batch_mask(batch_data, select_sample)

In [ ]:
def plot_prediction_mask(pred_mask, select_sample):
    plt.title("mask prediction")
    plt.imshow(np.argmax(pred_mask[select_sample].cpu().detach().numpy(), 0), cmap="gray")
    return

plot_prediction_mask(predictions, select_sample)

In [ ]:
calc_iou_per_class(predictions, batch_data.val_data_masks)